# Data Exploration
This document explores the raw data to determine what must be cleaned before modelling can begin. The data initialisation notebook was made in parallel with this document, exposing the inconsistencies and areas for improvement in the raw dataset.

In [1]:
# Perform necessary imports.
import pandas as pd
from lib.constants import *

## Exploring Match Data
In this section, we will explore the raw match data.

In [2]:
# Load match data.
match_data = pd.read_csv(DATA_PATH + "/Matches.txt", delimiter="\t")

1. Exploring the columns of the match data.

In [3]:
# Print the columns of match data.
match_data.columns

Index(['Match Id', 'Season Id', 'Season', 'Series Id', 'Series',
       'Series Gender Id', 'Series Gender', 'Match Date', 'Match YYMMDD',
       'Match Type Id', 'Match Type', 'Ball Type Id', 'Ball Type', 'TeamA Id',
       'TeamA', 'TeamA At Home', 'TeamB Id', 'TeamB', 'TeamB At Home',
       'Day/Night', 'Venue Id', 'Venue', 'Toss Won By Id', 'Toss Decision Id',
       'TeamA Innings1 Closure', 'TeamA Innings2 Closure',
       'TeamB Innings1 Closure', 'TeamB Innings2 Closure',
       'TeamA 1st Comparison', 'TeamA Result Id', 'TeamA Result',
       'TeamBattingIdMatchInnings1', 'TeamBattingMatchInnings1',
       'TeamBattingIdMatchInnings2', 'TeamBattingMatchInnings2',
       'TeamBattingIdMatchInnings3', 'TeamBattingMatchInnings3',
       'TeamBattingIdMatchInnings4', 'TeamBattingMatchInnings4',
       'TeamB Result Id', 'TeamB Result', 'TeamA Coach Id',
       'TeamA Coach Surname', 'TeamA Coach Other Names', 'TeamB Coach Id',
       'TeamB Coach Surname', 'TeamB Coach Other Name

From this, and for the purpose of this project, it appears that the Official's columns is redundant and could be removed for the purpose of data management.

2. Explore series gender.

In [4]:
# Print series gender.
match_data["Series Gender"].unique()

array(['Male', 'Female'], dtype=object)

In [5]:
# Print series gender ID.
match_data["Series Gender Id"].unique()

array([1, 2], dtype=int64)

For this project, we only need to focus on the male formats of cricket as we are attempting to model their performance. For the reasoning behind this can be read in the overview document.

3. Explore teams.

In [6]:
# Print all recorded teams.
match_data["TeamA"].unique()

array(['Australia (M)', 'West Indies (M)', 'Pakistan (M)',
       'Sri Lanka (M)', 'South Africa (M)', 'England (M)',
       'New Zealand (M)', 'India (M)', 'SA (M)', 'Tas (M)', 'NSW (M)',
       'Victoria (M)', 'WA (M)', 'Aus Intellectual Disability', 'Qld (M)',
       'Australia (F)', 'WA (F)', 'Qld Fire (F)', 'New Zealand (F)',
       'NSW Breakers (F)', 'Southern Scorpions (F)', 'Australia A (M)',
       'Zimbabwe (M)', 'Kenya (M)', 'ACT Meteors (F)', 'Victoria (F)',
       'England (F)', 'Scotland (M)', 'Bangladesh (M)', 'Ireland (M)',
       'India (F)', 'West Indies (F)', 'Tasmania Tigers (F)',
       'Sydney Sixers (M)', 'Melbourne Stars (M)',
       'Adelaide Strikers (M)', 'Perth Scorchers  (M)',
       'Brisbane Heat (M)', 'Hobart Hurricanes (M)',
       'Melbourne Renegades (M)', 'Sydney Thunder (M)', 'Canada (M)',
       'Gloucestershire (M)', 'South Africa Women (F)', 'Afghanistan (M)',
       'India A (M)', 'South Africa A (M)', 'Pakistan (F)',
       'Sri Lanka (F)', 'S

The only team that appears to need to be removed from this list is "Aus Intellectual Disability". The reasoning for removing this team can be read about in the overview document.

4. Check international games.

In [7]:
# Check for international games where Australia is not playing.
match_data[match_data.Series.str.contains("International") & ~match_data.TeamA.str.contains("Australia") & ~match_data.TeamB.str.contains("Australia")][["TeamA", "TeamB"]]

,TeamA,TeamB
19,West Indies (M),Zimbabwe (M)
75,Sri Lanka (M),England (M)
76,Sri Lanka (M),England (M)
77,Sri Lanka (M),England (M)
78,South Africa (M),Sri Lanka (M)
...,...,...
4018,India (M),England (M)
4019,India (M),England (M)
4020,India (M),England (M)
4021,India (M),England (M)


In [8]:
# Check number of international games where Australia is not playing.
len(match_data[match_data.Series.str.contains("International") & ~match_data.TeamA.str.contains("Australia") & ~match_data.TeamB.str.contains("Australia")][["TeamA", "TeamB"]])

429

5. Explore match format length.

In [9]:
# Print match types.
match_data["Match Type"].unique()

array(['5 Day', '1 Day', '4 Day', 'Twenty20', '3 Day', '2 Day'],
      dtype=object)

In [10]:
# Print match type Ids.
match_data["Match Type Id"].unique()

array([5, 1, 4, 7, 3, 2], dtype=int64)

At the top level, T20, 1 Day, and 5 Day are the only formats played consistently. However, domestically, 4 Day formats are preferred over 5 day. Hence, we will only keep T20, 1 Day, 4 Day, and 5 Day formats.

6. Check for columns containing NaN.

In [11]:
# Print columns containing NaN.
match_data.columns[match_data.isna().any()].tolist()

['TeamBattingMatchInnings1',
 'TeamBattingMatchInnings2',
 'TeamBattingMatchInnings3',
 'TeamBattingMatchInnings4',
 'TeamA Coach Id',
 'TeamA Coach Surname',
 'TeamA Coach Other Names',
 'TeamB Coach Id',
 'TeamB Coach Surname',
 'TeamB Coach Other Names',
 'Data Source',
 'Official1 Id',
 'Official1 Surname',
 'Official1 Other Names',
 'Official2 Id',
 'Official2 Surname',
 'Official2 Other Names',
 'Official3 Id',
 'Official3 Surname',
 'Official3 Other Names',
 'Official4 Id',
 'Official4 Surname',
 'Official4 Other Names',
 'Official5 Id',
 'Official5 Surname',
 'Official5 Other Names',
 'Official6 Id',
 'Official6 Surname',
 'Official6 Other Names']

From this, we can see we have already removed the majority of these fields by removing Officials. While it could be reasonable to remove other fields such as "Data Source" or Coach fields, we will leave these for now.

7. Check for duplicate rows.

In [12]:
# Print duplicate rows.
match_data[match_data.duplicated(keep=False)]

,Match Id,Season Id,Season,Series Id,Series,Series Gender Id,Series Gender,Match Date,Match YYMMDD,Match Type Id,...,Official3 Other Names,Official4 Id,Official4 Surname,Official4 Other Names,Official5 Id,Official5 Surname,Official5 Other Names,Official6 Id,Official6 Surname,Official6 Other Names


So, there are no duplicate rows.

8. Explore series.

In [13]:
# Print series out.
match_data.Series.unique()

array(['International Tests M', 'International ODI M',
       'Domestic 1st Class M', 'Domestic List A M', 'International T20 M',
       'Domestic T20 M', 'International ICC Trophy M',
       'International ODI World Cup M', 'International ODI F',
       'International T20 F', 'Domestic OD F',
       'International T20 World Cup M', 'Domestic T20 F',
       'International ODI World Cup F', 'International T20 World Cup F',
       'International Tests F', 'International 1st Class M',
       'International 1st Class F'], dtype=object)

## Exploring Delivery Data
In this section, we will explore the raw deliveries data.

To begin with, we must load the deliveries data. However, we will load the file in chunks, removing any delivery data not related to the curated set of matches outlined above.

In [2]:
# Load cleaned match data.
match_data = pd.read_csv(DATA_PATH + "/Matches_Clean.txt", delimiter="\t")

# Determine which matches are important for delivery data.
match_ids = match_data["Match Id"]

# Determine duplicate columns between match and delivery data that should be dropped.
match_columns = set(match_data.columns)
match_columns.remove("Match Id")

# Load delivery data
delivery_data = pd.DataFrame()

for chunk in pd.read_csv(DATA_PATH + "/Deliveries.txt", delimiter="\t", chunksize=10**6):
  chunk = chunk[chunk["Match Id"].isin(match_ids)]
  chunk.drop(
    [col for col in chunk.columns if col in match_columns], axis=1, inplace=True
  )

  # Combine filtered deliveries into single dataframe.
  delivery_data = pd.concat([delivery_data, chunk])

C:\Users\AlexK\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3377: DtypeWarning: Columns (6,19,85,87,101,103,200,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,233,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,396) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\AlexK\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3377: DtypeWarning: Columns (6,19,56,85,87,91,101,103,169,200,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,233,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,266,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318

1. Explore delivery columns.

In [4]:
# Print delivery columns.
list(delivery_data.columns)

['Match Id',
 'Team Batting At Home',
 'Team Bowling At Home',
 'Toss Won By Team',
 'Toss Won By Batting Team',
 'Toss Decision',
 'TeamA ResultId',
 'Team Batting Id',
 'Team Batting',
 'Team Bowling Id',
 'Team Bowling',
 'Innings',
 'Delivery',
 'Day',
 'Session',
 'Time of Day (Hour)',
 'Time of Day (Min)',
 'Time of Day',
 'Striker Id',
 'Striker',
 'Striker Hand Id',
 'Striker Hand',
 'Non Striker Id',
 'Non Striker',
 'Non Striker Hand Id',
 'Non Striker Hand',
 'Bowler Id',
 'Bowler',
 'Bowler Hand Id',
 'Bowler Hand',
 'Pace / Spin',
 'Bowler Style',
 'Spell',
 'Over The Wicket',
 'Northern End',
 'Power Play',
 'Over',
 'Ball In Over',
 'Fair Ball In Over',
 'Ball Speed',
 'Ball RPM',
 'Pitch X',
 'Pitch Y',
 'At Batter X',
 'At Batter Y',
 'At Stumps X',
 'At Stumps Y',
 'Hit To Len',
 'Hit To Angle',
 'Bat Score',
 'ReachedBoundary',
 'Wides',
 'Noballs',
 'Byes',
 'Legbyes',
 'Penalty Runs',
 'Taken By WK',
 'Batter Out Id',
 'Batter Out',
 'How Out',
 'Fielder1 Id',
 'Fi

We can't really see much from this. Instead, we will check for columns that contains NaN.

2. Check for columns containing NaN.

In [5]:
# Check columns that contain NaN
delivery_data.columns[delivery_data.isna().any()].tolist()

['Toss Won By Team',
 'Ball Speed',
 'Ball RPM',
 'Batter Out Id',
 'Batter Out',
 'How Out',
 'Fielder1 Id',
 'Fielder1',
 'Fielder2 Id',
 'Fielder2',
 'Fielder3 Id',
 'Fielder3',
 'Fielder4 Id',
 'Fielder4',
 'Fielder5 Id',
 'Fielder5',
 'Wind Description',
 'Event Grade',
 'Event Infield',
 'Fielder1 Catch',
 'Fielder1 Catch Assist',
 'Fielder1 Dropped Catch',
 'Fielder1 Runout',
 'Fielder1 Runout Missed',
 'Fielder1 Runout Assist',
 'Fielder1 Runout Assist Missed',
 'Fielder1 Missed Stumping',
 'Fielder1 Extra Effort',
 'Fielder1 Pressure Field',
 'Fielder1 Assist',
 'Fielder1 Fumble',
 'Fielder1 Misfield',
 'Fielder1 Dive Stop',
 'Fielder1 Dive Misfield',
 'Fielder1 Slide Stop',
 'Fielder1 Slide Miss',
 'Fielder1 Throw',
 'Fielder1 Good Throw',
 'Fielder1 Error Throw',
 'Fielder1 Throw Hit',
 'Fielder1 Throw Miss',
 'Fielder1 Throw Backed Up',
 'Fielder1 Throw Not Backed Up',
 'Fielder1 Keeper Drop',
 'Fielder1 Keeper Fumble',
 'Fielder1 Keeper Dive Stop',
 'Fielder1 Keeper Missed

3. Check for duplicate rows.

In [6]:
# Print duplicate rows.
delivery_data[delivery_data.duplicated(keep=False)]

,Match Id,Team Batting At Home,Team Bowling At Home,Toss Won By Team,Toss Won By Batting Team,Toss Decision,TeamA ResultId,Team Batting Id,Team Batting,Team Bowling Id,...,Movement Off Pitch,Stump Speed,Shot Aggression,Shot Quality Description,Hit To X Physical,Hit To Y Physical,Video File Name,Video Mark In Milliseconds,Keeper Id,Keeper


As there are no duplicate rows, there is no reason to perform any cleaning for this step.

4. Explore players that have played international one-day games.

In [5]:
# Determine IDs of international One-Day games.
odi_match_IDs = match_data[match_data.Series.str.contains("International") & match_data["Match Type Id"] == 1]["Match Id"]

# Extract deliveries within international One-Day games.
odi_deliveries = delivery_data[delivery_data["Match Id"].isin(odi_match_IDs)]

# Count number of international One Day games per player.
by_columns = ["Striker"]
aggregates = {"Match Id": pd.Series.nunique}
groupby_data = odi_deliveries.groupby(by=by_columns).agg(aggregates)
groupby_data

,Match Id
Striker,
"Aamer, Mohammad",2
"Abbott, Kyle",3
"Abbott, Sean",3
"Adams, Andre",2
"Adams, Jimmy",5
...,...
"van Schoor, Melt",1
"van Troost, Luuk",2
"van der Dussen, Rassie",1


Here we see two important factors that must be cleaned. Firstly, we are not interested in the batting careers of non-Australian players, so, we should strip the dataset to only include Australia's batting innings. Secondly, many players have only played a few innings. This is not reasonable to determine a players international performance, so, we must remove players that have played fewer than a threshold number of innings - 10 for example.

5. Explore the batting teams.

In [4]:
# Print all batting teams.
delivery_data["Team Batting"].unique()

array(['Pakistan (M)', 'Australia (M)', 'South Africa (M)',
       'West Indies (M)', 'Zimbabwe (M)', 'Kenya (M)', 'New Zealand (M)',
       'Bangladesh (M)', 'Sri Lanka (M)', 'England (M)', 'SA (M)',
       'Victoria (M)', 'NSW (M)', 'Tas (M)', 'WA (M)', 'Qld (M)',
       'India (M)', 'Ireland (M)', 'Scotland (M)', 'Netherlands (M)',
       'Australia A (M)', 'USA (M)', 'ICC World XI (M)', 'Namibia (M)',
       'Brisbane Heat (M)', 'Sydney Sixers (M)', 'Melbourne Stars (M)',
       'Sydney Thunder (M)', 'Adelaide Strikers (M)',
       'Melbourne Renegades (M)', 'Hobart Hurricanes (M)',
       'Perth Scorchers  (M)', 'Canada (M)', 'South Africa A (M)',
       'India A (M)', 'Afghanistan (M)', 'England Lions (M)', 'CA XI (M)',
       'Board President XI (M)', 'India B (M)'], dtype=object)

This demonstrates the issue of many non-Australian players being recorded.

6. Explore players that have played for multiple countries.

In [9]:
# Count number of international teams per player.
by_columns = ["Striker"]
aggregates = {"Team Batting": pd.Series.nunique}
groupby_data = odi_deliveries.groupby(by=by_columns).agg(aggregates)

# Extract players that have played for multiple countries.
multiple_country_batters = groupby_data[groupby_data["Team Batting"] > 1]

# Display players that have played for multiple countries.
multiple_country_batters

,Team Batting
Striker,
"Abbott, Kyle",2
"Abbott, Sean",2
"Afridi, Shahid",2
"Agar, Ashton",2
"Agarwal, Mayank",4
...,...
"Wade, Matthew",2
"White, Cameron",2
"Zampa, Adam",2


In [11]:
odi_deliveries[odi_deliveries["Striker"] == "Agarwal, Mayank"]["Team Batting"].unique()

array(['India A (M)', 'Board President XI (M)', 'India B (M)',
       'India (M)'], dtype=object)

As we can see, a player can play for multiple teams. Often, this is a player playing for different levels of international cricket for the one country. This is not necessarily the same for all players - for example, Luke Ronchi has played for NZ and AUS. We can address this issue by removing deliveries that are not bowled to an Australian team, rather than removing deliveries to a player that has played for multiple international teams. In the case of Luke Ronchi, this would mean removing his NZ performances, but not Australian.